In [5]:
import tensorflow as tf

In [7]:
import tensorflow_hub as hub

In [9]:
import tensorflow_datasets as tfds

In [11]:
import numpy as np

In [79]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)


In [15]:
import tensorflow_datasets as tfds
import time

for attempt in range(5):
    try:
        train_data, validation_data, test_data = tfds.load(
            name="imdb_reviews",
            split=('train[:60%]', 'train[60%:]', 'test'),
            as_supervised=True
        )
        break  # Exit the loop if successful
    except Exception as e:
        print(f"Attempt {attempt + 1} failed: {e}")
        time.sleep(5)  # Wait before retrying
else:
    print("Failed to download the dataset after 5 attempts.")


In [17]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [19]:
train_example_data, train_label_data = next(iter(train_data.batch(10))

SyntaxError: incomplete input (3864306513.py, line 1)

In [21]:
train_example_data, train_label_data = next(iter(train_data.batch(10)))


In [23]:

print("GPu is"," available" if tf.config.experimental.list_physical_devices("GPU") else "NOT available")

GPu is NOT available


In [25]:
train_label_data

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

In [157]:
import tensorflow_hub as hub
import tensorflow as tf

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer= hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

In [159]:
hub_layer(train_example_data[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [167]:
model= tf.keras.Sequential()
model.add(hub_layer)


ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x0000026145DFB450> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

In [177]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras import layers

# Custom layer that wraps the TensorFlow Hub layer
class HubLayer(layers.Layer):
    def __init__(self, hub_url):
        super(HubLayer, self).__init__()
        self.hub_layer = hub.KerasLayer(hub_url, dtype=tf.string, trainable=True)

    def call(self, inputs):
        return self.hub_layer(inputs)

# Define the embedding URL
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

# Use the Functional API
inputs = tf.keras.Input(shape=(), dtype=tf.string)
x = HubLayer(embedding)(inputs)
x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Summary of the model
model.summary()


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None)                 │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hub_layer_9 (HubLayer)          │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 16)             │           336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 353 (1.38 KB)

 Trainable params: 353 (1.38 KB)

 Non-trainable params: 0 (0.00 B)

In [179]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [181]:
history=model.fit(train_data.shuffle(1000).batch(100),epochs=25,
                  validation_data=validation_data.batch(100),verbose=1)

Epoch 1/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5572 - loss: 0.7599 - val_accuracy: 0.6485 - val_loss: 0.6136
Epoch 2/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6567 - loss: 0.6096 - val_accuracy: 0.6688 - val_loss: 0.5921
Epoch 3/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6752 - loss: 0.5895 - val_accuracy: 0.6582 - val_loss: 0.5878
Epoch 4/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6776 - loss: 0.5805 - val_accuracy: 0.6934 - val_loss: 0.5736
Epoch 5/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6878 - loss: 0.5737 - val_accuracy: 0.6888 - val_loss: 0.5677
Epoch 6/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6884 - loss: 0.5724 - val_accuracy: 0.7020 - val_loss: 0.5661
Epoch 7/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6922 - loss: 0.5684 - val_accuracy: 0.6933 - val_loss: 0.5609
Epoch 8/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6869 - loss: 0.5649 - val_accuracy: 0.

In [189]:
results= model.evaluate(test_data.batch(100), verbose=2)
for name, values in zip(model.metrics_names,results):
    print("%s:%.3f"%(name, values))

250/250 - 1s - 4ms/step - accuracy: 0.6871 - loss: 0.5553
loss:0.555
compile_metrics:0.687
